Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and student ID below:

In [ ]:
NAME1 = ""
NAME2 = ""
NAME3 = ""
ID1 = "" ## Your student id
ID2 = ""
ID3 = ""

---

# Lab 2 Play with sound

Here we begin our acoustic sensing journey. In the first stage, you have to be familar with your speaker and microphones in the code level. We will introduce `sounddevice` and guide you record your own voice.

Before we start, you should have installed all the packages and modules needed. If not, you can refer to Lab 0. If you are running with errors like `ImportError: No module named ...`, you should check the installation status of the corresponding packages. If not yet, you can run `pip install <module-name>` in your command line. If you have installed and encounter this problem, check if they are in your environment path. You are encouraged to seek help from Google or your teammates.  

In [ ]:
import numpy as np
import sounddevice as sd
# You may need run `conda install -c conda-forge libsndfile` if you use Mac M1
import soundfile as sf
from scipy import signal
from matplotlib import pyplot as plt

We use `sounddevice` as our control of speakers and microphones. This Python module provides bindings for the PortAudio library and a few convenience functions to play and record NumPy arrays containing audio signals. It is available for Linux, macOS and Windows.

You can refer to its document by clicking [Sounddevice document](https://python-sounddevice.readthedocs.io/)

## Know what is on your device

You can use `query_devices` to see all the audio devices:

In [ ]:
sd.query_devices()

There are some default settings of what `sounndevice` will do. These settings are stored in `class sounddevice.default`. It contains attributes including `device`, `channels`, `dtype`, `latency` and `extra_settings`. They accept single values which specify the given property for both input and output. However, if the property differs between input and output, pairs of values can be used, where the first value specifies the input and the second value specifies the output. All other attributes are always single values.

In [ ]:
fs = (int)(48e3)
latency = 'high','high'
channel = 1,1
device_in = 'MacBook Pro Microphone' # Change according to your device
device_out = 'MacBook Pro Speakers'
dtype = np.float32

sd.default.latency = latency
sd.default.samplerate = fs
sd.default.channels = channel
sd.default.device = device_in, device_out
sd.default.dtype = dtype

You can check whether these settings are compatabile with your devices by calling `check_input_settings()` and `check_output_settings()`

In [ ]:
print(sd.check_input_settings())
print(sd.check_output_settings())

If the output is `None`, it indicates your settings are correct.

In [ ]:
assert(sd.check_input_settings()==None)
assert(sd.check_output_settings()==None)

## Play and record a clip 

After setting your hardwares, you may be curious about how to use them. In this part, we will introduce some APIs, which could play and record the sounds using the sound device. 

Check your directory, you will see a `.wav` file called `canon.wav`. We will use this audio as an example to show how to play and record audios.

In [ ]:
filename = './canon.wav'
data,fs = sf.read(filename,dtype='float32')
one_channel_data = data[:,0]
# sd.play(one_channel_data,fs)

We only want one channel of the sound. That is what `one_channel_data` do. We use `sf.read()` to read a audio file and it will be written into `data` as numpy array. `fs` is the sampling rate of the wav, which is 48000Hz. `sd.play()` is called to play the sounds. If you want to stop it, you call call

In [ ]:
sd.stop()

We can see what we are playing:

In [ ]:
one_channel_data

In [ ]:
one_channel_data.reshape(-1,1).shape

See, The sound are complex values. We can plot it in time-domain.

In [ ]:
t = np.arange(0, one_channel_data.shape[0] / fs, 1/fs)
plt.plot(t, one_channel_data)
plt.xlim(0,np.max(t))

It is hard to acquire useful information from this time-domain data. How about frequency domain?

In [ ]:
data_fft = np.fft.fft(one_channel_data)
data_fft_len = (int)(data_fft.shape[0] / 2)
data_fft_freq = np.fft.fftfreq(data_fft.shape[0], d = 1/fs)
data_abs = np.abs(data_fft)
plt.plot(data_fft_freq[:data_fft_len],data_abs[:data_fft_len])

The main frequencies range mostly less than 5kHz. In common sense, human can hear sounds from 20 to 2kHz. So, that is exactly what we expect. You can also use `sd.record()` to record your sounds. How about recording the playing audios? For simple data, `sounddevice` give us a simple solution: `playrec`. By running it, you can record and play simultaneously.

In [ ]:
myrecording = sd.playrec(data, fs)

You can check your recording by running

In [ ]:
sd.play(myrecording)

## Streams and Callback Function

Up to now, you may have learnt how to play and record the sounds. It sounds great since you are getting familar with your hardware device. However, the aforementioned functions are designed for small scripts and not enough to implement an acoustic sensing task. Under most circumstances, we need more control over these data, like continuous recording as well as real time processing. 

Therefore, we introduce `Stream`, which is a more low-level class for audio controls. When a stream is running, PortAudio calls the  `callback()` periodically. The callback function is responsible for processing and filling input and output buffers, respectively. 

Here is a typical `callback()` signature:
```
callback(indata: ndarray, outdata: ndarray, frames: int,time: CData, status: CallbackFlags) -> None
```

`indata` is what the microphone receives while `outdata` is what the speaker plays. Frames refer to the number of frames to be processed by the stream callback. This is the same as the length of the input and output buffers. `status` is a CallbackFlags instance indicating whether input and/or output buffers have been inserted or will be dropped to overcome underflow or overflow conditions. More information could be found by reading the `sounddevice` document.

<b><font color="red" size=5>Checkpoints (15 points)</font></b>

Implement the `callback` function. You need to make sure the shape of the recorded data is the SAME as that of the input audio `one_channel_data`. You should store your recordings in `datarec`, which is a global variable.

In [ ]:
'''
    callback(indata: ndarray, outdata: ndarray, frames: int,time: CData, status: CallbackFlags) -> None
    - Input
        * indata  : recorded data
        * outdata : playing data
        * frames  : input and output buffers
        * time    : a CFFI structure with timestamps
        * status  : underflow/overflow flags
    - Function: 
        * Control the indata and outdata; The outdata should be `one_channel_data`, and indata should 
          be the recording data;
        * You should store your recordings in `datarec`
        * The shape of `datarec` should be the same as `one_channel_data`
'''
idx = 0 
datarec = np.array([]).reshape(-1,1)
data_len = len(one_channel_data)
def callback(indata, outdata, frames, time, status):
    global datarec
    if status:
        print(status)
    
    # YOUR CODE HERE
    raise NotImplementedError()

try:
    with sd.Stream(callback=callback):
        # YOUR CODE HERE
        raise NotImplementedError()
except sd.CallbackStop:
    exit('')
except KeyboardInterrupt:
    exit('')
except Exception as e:
    exit(type(e).__name__ + ': ' + str(e))

By now you should know how to work with your sound devices and learn to record the sounds being played. You will probably use them in the next tasks.

Alternatively, you can use your existing knowledge to make a live recording of the device and display its spectrum. This is not compulsory. But if you accomplish it, you can earn some bonus points.

<b><font  size=3>Bonus Task (optional)</font></b>

Making a live recording and depict the spectrum.